In [2]:
import sys
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
# import cell2location
import scvi
import os
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42
import seaborn as sns
import time
import gc
gc.collect()


39207

In [1]:

import cell2location

/home/myi/data/conda/envs/c2l/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/myi/data/conda/envs/c2l/lib/python3.9/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/myi/data/conda/envs/c2l/lib/python3.9/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (


In [ ]:
# load data
cellcount = "/home/myi/data/scRNA-seq/sc_counts_SHK166_RA_Knee.mtx"
sp_data = "/home/myi/data/spatial-trancriptomics/st_counts_SHK166_RA_Knee.mtx"
celltype = "/home/myi/data/scRNA-seq/ctypes7_counts_SHK166_RA_Knee.csv"

start_time = time.time()
adata_ref = sc.read_mtx(cellcount).transpose()
df_celltype = pd.read_csv(celltype, header=0, index_col=0)
df_celltype.index = adata_ref.obs.index
adata_ref.obs['Subset'] = df_celltype
shuffled_obs_indices = np.random.permutation(adata_ref.n_obs)
adata_ref = adata_ref[shuffled_obs_indices, :]
print(df_celltype)
print(adata_ref.obs.index)

In [ ]:
!pwd
adata_file = "./results/analysis/reference_signatures/sc.h5ad"
adata_vis = sc.read_h5ad(adata_file)

In [ ]:
# select one slide
from cell2location.utils import select_slide
slide = select_slide(adata_vis, 'SHK')


In [ ]:
start_time = time.time()
adata_ref = sc.read_mtx(cellcount).transpose() # cells by genes
df_celltype = pd.read_csv(celltype, header=0, index_col=0)
print(df_celltype)
df_celltype.index = adata_ref.obs.index
adata_ref.obs['Subset'] = df_celltype

df_celltype.head()
# Get 'Method'
adata_ref.obs['Method'] = '3GEX'

# Get 'Sample'
adata_ref.obs['Sample'] = adata_ref.obs_names
adata_ref.obs['Sample'] = adata_ref.obs['Sample'].apply(lambda x: x[0:4])

In [ ]:
from cell2location.utils.filtering import filter_genes
selected = filter_genes(adata_ref, cell_count_cutoff=5, cell_percentage_cutoff2=0.03, nonz_mean_cutoff=1.12)
# 5, 0.03, 1.12

# In our case, a few genes are cut
adata_ref = adata_ref[:, selected].copy()

In [ ]:
scvi.data.setup_anndata(adata=adata_ref,
                        # 10X reaction / sample / batch
                        batch_key='Sample',
                        # cell type, covariate used for constructing signatures
                        labels_key='Subset',
                        # multiplicative technical effects (platform, 3' vs 5', donor effect)
                        categorical_covariate_keys=['Method']
                       )
scvi.data.view_anndata_setup(adata_ref)

In [ ]:
from cell2location.models import RegressionModel
mod = RegressionModel(adata_ref)

# Use all data for training (validation not implemented yet, train_size=1)
mod.train(max_epochs=250, batch_size=2500, train_size=1, lr=0.002, use_gpu=True)

# plot ELBO loss history during training, removing first 20 epochs from the plot
mod.plot_history(20)

In [ ]:
# In this section, we export the estimated cell abundance (summary of the posterior distribution).
adata_ref = mod.export_posterior(
    adata_ref, sample_kwargs={'num_samples': 1000, 'batch_size': 2500, 'use_gpu': True}
)

# Results saving folder
ref_run_name = f'./results/analysis/reference_signatures'

# Save model
mod.save(f"{ref_run_name}", overwrite=True)

adata_file = f"{ref_run_name}/sc.h5ad"
adata_ref.write(adata_file)
adata_file

In [ ]:
ç

In [ ]:
# Export estimated expression in each cluster
if 'means_per_cluster_mu_fg' in adata_ref.varm.keys():
    inf_aver = adata_ref.varm['means_per_cluster_mu_fg'][[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
else:
    inf_aver = adata_ref.var[[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
inf_aver.columns = adata_ref.uns['mod']['factor_names']
inf_aver.iloc[0:5, 0:5]

# Note that the column names are cell types, the row names are gene names, in the original code
# the authors use ENSEMBL as names instead of raw gene names

In [ ]:
adata_vis = sc.read_csv(sp_data)
adata_vis.obs['sample'] = 'pseudo_st' # Since it is manually generated

In [ ]:
intersect = np.intersect1d(adata_vis.var_names, inf_aver.index)
adata_vis = adata_vis[:, intersect].copy()
inf_aver = inf_aver.loc[intersect, :].copy()

In [ ]:
scvi.data.setup_anndata(adata=adata_vis, batch_key="sample")
scvi.data.view_anndata_setup(adata_vis)

In [ ]:
gc.collect()
mod = cell2location.models.Cell2location(
    adata_vis, cell_state_df=inf_aver,
    # the expected average cell abundance: tissue-dependent
    # hyper-prior which can be estimated from paired histology:
    N_cells_per_location=6,
    # hyperparameter controlling normalisation of
    # within-experiment variation in RNA detection (using default here):
    detection_alpha=200
)

mod.train(max_epochs=20000,
          # train using full data (batch_size=None)
          batch_size=None,
          # use all data points in training because
          # we need to estimate cell abundance at all locations
          train_size=1,
          use_gpu=True)

# plot ELBO loss history during training, removing first 100 epochs from the plot
mod.plot_history(1000)
plt.legend(labels=['full data training']);

In [ ]:
# In this section, we export the estimated cell abundance (summary of the posterior distribution).
adata_vis = mod.export_posterior(
    adata_vis, sample_kwargs={'num_samples': 1000, 'batch_size': mod.adata.n_obs, 'use_gpu': True}
)

In [ ]:
# Save model
run_name = f'./results/analysis/cell2location_map'

mod.save(run_name, overwrite=True)
end_time = time.time()

In [ ]:
# Save anndata object with results
adata_file = f"{run_name}/sp.h5ad"
adata_vis.write(adata_file)
adata_file

In [ ]:
# Examine reconstruction accuracy to assess if there are any issues with mapping
# the plot should be roughly diagonal, strong deviations will signal problems
mod.plot_QC()

In [ ]:
# add 5% quantile, representing confident cell abundance, 'at least this amount is present',
# to adata.obs with nice names for plotting
adata_vis.obs[adata_vis.uns['mod']['factor_names']] = adata_vis.obsm['q05_cell_abundance_w_sf']
result1 = adata_vis.obsm['q05_cell_abundance_w_sf']
result2 = adata_vis.obsm['q95_cell_abundance_w_sf']
result3 = adata_vis.obsm['means_cell_abundance_w_sf']
# result_mean = result3.to_csv('mean_gene_expressionhalfx.csv')
# result2.to_csv('95_gene_expressionhalfx.csv')
# result1.to_csv('05_gene_expressionhalfx.csv')

In [ ]:
sum_result_3 = result3.sum(axis=1)
result3_percent = result3.div(result3.assign(total=sum_result_3)['total'], axis='index')
result_name = os.path.join(parent_path, "seqFISH_10000_Result", "Cell2location.csv")
result3_percent.to_csv(result_name)

In [ ]:
sum_result_2 = result2.sum(axis=1)
result2_percent = result2.div(result2.assign(total=sum_result_2)['total'], axis='index')
result2_percent.to_csv('q95_gene_expression_test_SeqFISH_10000.csv')

In [ ]:
sum_result_1 = result1.sum(axis=1)
result1_percent = result1.div(result1.assign(total=sum_result_1)['total'], axis='index')
result1_percent.to_csv('q05_gene_expression_test_SeqFISH_10000.csv')

In [ ]:
print("Total time comsuption: seconds")
print(end_time - start_time)

In [ ]:
# stdata = pd.read_csv('stdata_MERFISH_20.csv', header=0, index_col=0)
# scdata = pd.read_csv('cellcount_MERFISH_20.csv', header=0, index_col=0)
# print(stdata.shape)
# print(scdata.shape)